<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/sql_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate SQL according Meta Data of Database

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** None

**Description:**

Answering questions from bosses or coworkers about business with unprepared data is too often a situation for a business analyst or a data scientist. Sometimes it takes too much time and chops your work time into too small chunks. But what more can you do? Bosses and coworkers cannot write SQL for themselves, so they are counting on you!

But what if you have an agent to write SQL from natural language questions? Does that help you a lot? Take a look at this showcase and explore how Agently agent can help.

作为一个商业分析师或者数据科学家，你是否经常遇到你的老板、同事在不停地问你一些数据报表中根本没有预处理的数据？作为一个分析研究的专家，你当然想把时间花在“分析”而不是“提数”上，而且总是处理“提数”这件事可能把你深度思考的时间切得粉碎。但你能怎么办呢？你的老板、同事又不会写SQL，他们只能找你这个“专家”。

如果有一个Agent，能够帮你把自然语言问题转换成SQL，是不是就帮了大忙了？看看下面这个案例是如何使用Agently框架，写几行简单的代码解决这个问题的。

## Step 1: Install Packages

In [ ]:
!pip install -U Agently

## Step 2: Demo Code

### Information of Data Tables

We got a MySQL database with two tables like this:

|||Table 1: user_info|||
---|---|---|---|---
column name | column desc | data type | data example | annotation
user_id | user identity code | string | "u_123456" |
gender | user gender | int | 0 | 0: female, 1: male
age | user age | int | 18 |
customer_level | user's customer level | int | 4 | 1: copper, 2: silver, 3: gold, 4: platinum, 5: vvip
reg_time | when did user register | timestamp | 1701340135721 |
reg_channel | which channel did user come from | int | 0 | 0: natural, 1: game ads, 2: video ads, 3: streaming live ads, 4: offline post ads

|||Table 2: order |||
---|---|---|---|---
column name | column desc | data type | data example | annotation
order_id | order identity code | string | "o_456789" |
customer_user_id | user identity of who paid this order | string | "u_234567" |
item_name | item's name | string | "HD Vanilla Icecream" |
item_class_tags | tags of item | JSON String | ["food", "snack", "frozen"] | standard JSON string format
price | price of 1 item | float | 1.35 | unit: $
order_item_number | how many items in this order | int | 3 |
order_time | when was this order created | timestamp | 1701341789164 |
pay_time | when was this order paid | timestamp | 1701341835323 | value should be 0 if not paid

As you can see, those two tables are very common in E-commerce system.

### DEMO: Generate SQL

In [12]:
import Agently
from datetime import datetime

agent_factory\
    .set_settings("model.OpenAI.auth", { "api_key": "" })

# Create a meta data dict of those two tables:
## Usually we can get the meta data from our system by requesting APIs
## instead of typing them down below manually
meta_data = [
    {
        "table_name": "user_info",
        "columns": [
            { "name": "user_id", "desc": "user identity code", "data type": "string", "example": "u_123456" },
            { "name": "gender", "desc": "user gender", "data type": "int", "example": 0, "annotation": "0: female, 1: male" },
            { "name": "age", "desc": "user age", "data type": "int", "example": 18 },
            { "name": "customer_level", "desc": "user's customer level", "data type": "int", "example": 4, "annotation": "1: copper, 2: silver, 3: gold, 4: platinum, 5: vvip" },
            { "name": "reg_time", "desc": "when did user register", "data type": "timestamp", "example": 1701340135721 },
            { "name": "reg_channel", "desc": "which channel did user come from", "data type": "int", "example": 0, "annotation": "0: natural, 1: game ads, 2: video ads, 3: streaming live ads, 4: offline post ads" },
        ],
    },
    {
        "table_name": "order",
        "columns": [
            { "name": "order_id", "desc": "order identity code", "data type": "string", "example": "o_456789" },
            { "name": "customer_user_id", "desc": "user identity of who paid this order", "data type": "string", "example": "u_234567" },
            { "name": "item_name", "desc": "item's name", "data type": "string", "example": "HD Vanilla Icecream" },
            { "name": "item_class_tags", "desc": "tags of item", "data type": "JSON String", "example": "[\"food\", \"snack\", \"frozen\"]", "annotation": "standard JSON string format" },
            { "name": "price", "desc": "price of 1 item", "data type": "float", "example": 1.35, "annotation": "unit: $" },
            { "name": "order_item_number", "desc": "how many items in this order", "data type": "int", "example": 3 },
            { "name": "order_time", "desc": "when was this order created", "data type": "timestamp", "example": 1701341789164 },
            { "name": "pay_time", "desc": "when was this order paid", "data type": "timestamp", "example": 1701341835323 },
        ],
    }
]
## As you can see, this `meta_data` dict has a complex structure
## But it's OK, because Agently framework can help you handle the complex structure
## You can pass the dict to .info() or .input() like a variable directly
## instead of trying to figure out how to prompt or how to make model understand

agent = agent_factory.create_agent()
print("Ask questions about data using natural language.\nInput '#exit' if you want to stop.\n------\n")
while True:
    user_input = input("[Question]: ")
    if user_input == "#exit":
        break
    result = agent\
        .input(user_input)\
        .info({ "database meta data": meta_data })\
        .info({ "current date": datetime.now().date() })\
        .instruct("Generate SQL for MySQL database according {database meta data} to answer {input}")\
        .output({
            "thinkings": [("String", "Your thinkings step by step about how to query data to answer {input}")],
            "SQL": ("String", "SQL String without explanation"),
        })\
        .start()
    print("[Thinking Process]: ")
    for thinking in result["thinkings"]:
        print(thinking)
    print("[SQL]: \n", result["SQL"])
    print("------\n")

Ask questions about data using natural language.
Input '#exit' if you want to stop.
------

[Question]: What's the user revenue of those who came because ads in games last month?
[Thinking Process]: 
We need to retrieve the revenue of users who came because ads in games last month.
To do this, we need to join the 'user_info' and 'order' tables on the user identity code.
We also need to filter the orders based on the 'reg_channel' column from the 'user_info' table, where the value is 1 (game ads) and the 'pay_time' column from the 'order' table is within the last month.
We can calculate the revenue by multiplying the 'price' column with the 'order_item_number' column in the 'order' table.
Finally, we need to group the revenue by user and sum it to get the total revenue of those users.
[SQL]: 
 SELECT ui.user_id, SUM(o.price * o.order_item_number) AS revenue FROM user_info ui JOIN order o ON ui.user_id = o.customer_user_id WHERE ui.reg_channel = 1 AND o.pay_time >= DATE_SUB(CURDATE(), IN